In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, BatchNormalization, Conv2D, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import pandas as pd
import Functions
import var
import Viz
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

[Link to implementation using Keras](https://engmrk.com/alexnet-implementation-using-keras/)

In [ ]:
def get_alexnet(dim):
    #Instantiate an empty model
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=dim, kernel_size=(11,11), strides=(4,4), padding='valid', 
                    activation = 'relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid', 
                    activation = 'relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', 
                    activation = 'relu'))

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', 
                    activation = 'relu'))

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', 
                    activation = 'relu'))
    
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

    # Passing it to a Fully Connected layer
    model.add(Flatten())
    
    # 1st Fully Connected Layer
    model.add(Dense(4096, input_shape=(227*227*3,), activation = 'relu'))
    
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.3))

    # 2nd Fully Connected Layer
    model.add(Dense(4096, activation = 'relu'))
    # Add Dropout
    model.add(Dropout(0.3))

    # 3rd Fully Connected Layer
    model.add(Dense(1000, activation = 'relu'))
    # Add Dropout
    model.add(Dropout(0.3))

    # Output Layer
    model.add(Dense(3, activation = 'softmax'))


    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
x_train, x_test, y_train, y_test = Functions.get_samples('alexnet')
print(x_train.shape, x_test.shape) 
print(y_train.shape, y_test.shape)

In [ ]:
augment = True 

early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=30)
model_checkpoint = ModelCheckpoint('../models/Alexnet-ModelCheckpointWeights.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 16
if var.img_type == 'grey': 
    dim = (var.alex_dimension[0], var.alex_dimension[1], 1)
else: 
    dim = (var.alex_dimension[0], var.alex_dimension[1], 3)
    
alexnet = get_alexnet(dim =dim)

if augment: 
    augmentation =ImageDataGenerator(rotation_range = 15, width_shift_range = .1, height_shift_range = .1, 
                                                           horizontal_flip = True, fill_mode = 'nearest')
    augmentation.fit(x_train)
    alexnet_history = alexnet.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
                epochs = epochs, steps_per_epoch = 8,
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)
else: 
    
    alexnet_history = alexnet.fit(x_train, y_train, batch_size = batch_size,
                epochs = epochs, 
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)


In [ ]:
print(alexnet.summary())

### Loss and Accuracy

In [ ]:
Viz.plot_loss_accuracy(alexnet_history)

### ROC and AUC

In [ ]:
if var.img_type == 'grey': 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 1)
else: 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
    
alexnet = get_alexnet(dim)
alexnet.load_weights('../models/Alexnet-ModelCheckpointWeights.h5') #load the best weights before overfitting

Viz.plot_roc_auc(alexnet, x_test, y_test)

### Confusion Matrix

In [ ]:
if var.img_type == 'grey': 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 1)
else: 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
    
mobilenet = get_mobile_net(dim)
mobilenet.load_weights('../models/MobileNet-ModelCheckpointWeights.h5') #load the best weights before overfitting





y_test_prob = [np.argmax(i) for i in mobilenet.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in mobilenet.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'])


In [ ]:
# assert False

### Test on Google

In [ ]:
import cv2
from lime import lime_image

In [ ]:
base_path = '../Tests/Photos'
if var.img_type == 'grey': 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 1)
else: 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
    
mobilenet = get_mobile_net(dim)
mobilenet.load_weights('../models/MobileNet-ModelCheckpointWeights.h5') #load the best weights before overfitting

for file in os.listdir(base_path): 
    if file == '.ipynb_checkpoints':
        continue
    full_path = f'{base_path}/{file}'
    img = Functions.get_image_value(full_path, var.mobilenet_dimension, var.img_type)
    img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
    pred = mobilenet.predict(img)[0]
    print(f'{file}\t\t{np.argmax(pred)}\t\t{pred.max()}\t\t{pred}')

In [ ]:
img = GetPickles.get_image_value('../TestImages/AR.jpg', var.mobilenet_dimension, var.img_type)

explainer = lime_image.LimeImageExplainer()

explanation = explainer.explain_instance(img, mobilenet.predict, top_labels = 5, hide_color = 0, 
                                         num_samples = 1000)

temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only = False,
                                           num_features = 10, hide_rest = False)
plt.imshow(mark_boundaries(temp/2 + .5, mask))

In [ ]:
Functions.get_prediction_bounding_box('../TestImages/AR.jpg', mobilenet, var.mobilenet_dimension)

In [ ]:
assert False